# Project 1 – Airline Arrival Delay Analysis

## Purpose
The goal of this project is to analyze arrival delay performance for two airlines,
Alaska and AM West, across five destinations. Using a CSV dataset and pandas,
I compare on-time versus delayed arrivals to determine which airline performs
better overall and whether performance differs by destination.

## Analysis Plan

To ensure a clear and structured analysis, I follow these steps:

1. Convert the provided arrival table into a CSV file so the data can be processed programmatically.
2. Load the CSV file into a pandas DataFrame and verify that the data was read correctly.
3. Restructure the data to calculate total arrivals and delay rates for each airline and destination.
4. Compare overall delay performance between airlines using a weighted delay rate.
5. Examine delay performance by destination to determine whether results differ by route.
6. Summarize findings and discuss any limitations of the dataset.

## Data Creation

The arrival data was originally provided as a table in the assignment instructions.
To analyze it using pandas, I first convert the table into a CSV file.
The data is structured in a tidy format, with one row per airline, destination,
and arrival status (on-time or delayed). This structure makes the data easier
to group, summarize, and analyze programmatically.

In [4]:
import pandas as pd

data = [
    # Alaska Airlines
    {"airline": "Alaska", "destination": "Los Angeles",   "status": "on_time", "count": 497},
    {"airline": "Alaska", "destination": "Los Angeles",   "status": "delayed", "count": 62},
    {"airline": "Alaska", "destination": "Phoenix",       "status": "on_time", "count": 221},
    {"airline": "Alaska", "destination": "Phoenix",       "status": "delayed", "count": 12},
    {"airline": "Alaska", "destination": "San Diego",     "status": "on_time", "count": 212},
    {"airline": "Alaska", "destination": "San Diego",     "status": "delayed", "count": 20},
    {"airline": "Alaska", "destination": "San Francisco", "status": "on_time", "count": 503},
    {"airline": "Alaska", "destination": "San Francisco", "status": "delayed", "count": 102},
    {"airline": "Alaska", "destination": "Seattle",       "status": "on_time", "count": 1841},
    {"airline": "Alaska", "destination": "Seattle",       "status": "delayed", "count": 305},

    # AM West Airlines
    {"airline": "AM West", "destination": "Los Angeles",   "status": "on_time", "count": 694},
    {"airline": "AM West", "destination": "Los Angeles",   "status": "delayed", "count": 117},
    {"airline": "AM West", "destination": "Phoenix",       "status": "on_time", "count": 4840},
    {"airline": "AM West", "destination": "Phoenix",       "status": "delayed", "count": 415},
    {"airline": "AM West", "destination": "San Diego",     "status": "on_time", "count": 383},
    {"airline": "AM West", "destination": "San Diego",     "status": "delayed", "count": 65},
    {"airline": "AM West", "destination": "San Francisco", "status": "on_time", "count": 320},
    {"airline": "AM West", "destination": "San Francisco", "status": "delayed", "count": 129},
    {"airline": "AM West", "destination": "Seattle",       "status": "on_time", "count": 201},
    {"airline": "AM West", "destination": "Seattle",       "status": "delayed", "count": 61},
]

df = pd.DataFrame(data)
df.to_csv("airline_arrivals.csv", index=False)

df.head()

,airline,destination,status,count
0,Alaska,Los Angeles,on_time,497
1,Alaska,Los Angeles,delayed,62
2,Alaska,Phoenix,on_time,221
3,Alaska,Phoenix,delayed,12
4,Alaska,San Diego,on_time,212


## Load and Validate the Data

After creating the CSV file, I load it back into pandas to confirm the file was
saved correctly and that the data types and rows match what I expect. This
validation step helps prevent analysis errors caused by missing data or incorrect
data types.

In [6]:
df = pd.read_csv("airline_arrivals.csv")

df.head(), df.tail()

(  airline  destination   status  count
 0  Alaska  Los Angeles  on_time    497
 1  Alaska  Los Angeles  delayed     62
 2  Alaska      Phoenix  on_time    221
 3  Alaska      Phoenix  delayed     12
 4  Alaska    San Diego  on_time    212,
     airline    destination   status  count
 15  AM West      San Diego  delayed     65
 16  AM West  San Francisco  on_time    320
 17  AM West  San Francisco  delayed    129
 18  AM West        Seattle  on_time    201
 19  AM West        Seattle  delayed     61)

In [7]:
df.info()
df["airline"].value_counts(), df["destination"].nunique(), df["status"].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   airline      20 non-null     object
 1   destination  20 non-null     object
 2   status       20 non-null     object
 3   count        20 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 772.0+ bytes


(airline
 Alaska     10
 AM West    10
 Name: count, dtype: int64,
 5,
 status
 on_time    10
 delayed    10
 Name: count, dtype: int64)

## Delay Rate Calculation

To fairly compare airline performance, I calculate a delay rate for each airline
and destination. Using delay rates instead of raw delayed counts allows for a
fair comparison because airlines may operate different numbers of flights on
each route.

In [9]:
pivot = df.pivot_table(
    index=["airline", "destination"],
    columns="status",
    values="count",
    aggfunc="sum",
    fill_value=0
).reset_index()

pivot["total_arrivals"] = pivot["on_time"] + pivot["delayed"]
pivot["delay_rate"] = pivot["delayed"] / pivot["total_arrivals"]

pivot.sort_values(["airline", "delay_rate"], ascending=[True, False])

status,airline,destination,delayed,on_time,total_arrivals,delay_rate
3,AM West,San Francisco,129,320,449,0.287305
4,AM West,Seattle,61,201,262,0.232824
2,AM West,San Diego,65,383,448,0.145089
0,AM West,Los Angeles,117,694,811,0.144266
1,AM West,Phoenix,415,4840,5255,0.078972
8,Alaska,San Francisco,102,503,605,0.168595
9,Alaska,Seattle,305,1841,2146,0.142125
5,Alaska,Los Angeles,62,497,559,0.110912
7,Alaska,San Diego,20,212,232,0.086207
6,Alaska,Phoenix,12,221,233,0.051502


## Delay Rate Comparison by Destination

The table above shows delay rates for each airline by destination.
Comparing delay rates instead of raw counts allows for a fair assessment
of performance, since airlines operate different numbers of flights on
each route. Lower delay rates indicate better on-time performance.

In [11]:
by_destination = pivot.pivot_table(
    index="destination",
    columns="airline",
    values="delay_rate"
)

by_destination["Difference (AM West - Alaska)"] = (
    by_destination["AM West"] - by_destination["Alaska"]
)

by_destination.sort_values("Difference (AM West - Alaska)")

airline,AM West,Alaska,Difference (AM West - Alaska)
destination,,,
Phoenix,0.078972,0.051502,0.027470
Los Angeles,0.144266,0.110912,0.033354
San Diego,0.145089,0.086207,0.058882
Seattle,0.232824,0.142125,0.090700
San Francisco,0.287305,0.168595,0.118710


### Interpretation

Across most destinations, Alaska Airlines has a lower delay rate than AM West,
indicating stronger on-time performance on a route-by-route basis. However,
performance differences vary by destination. In some locations, AM West
handles a much higher volume of flights, which can impact its overall delay
rate despite performing competitively on certain routes.

## Overall Delay Performance

To determine which airline performs better overall, I calculate a weighted
delay rate. This approach accounts for the total number of flights operated
by each airline, providing a more accurate overall comparison than averaging
route-level delay rates.

In [14]:
overall = pivot.groupby("airline")[["delayed", "total_arrivals"]].sum()
overall["overall_delay_rate"] = overall["delayed"] / overall["total_arrivals"]
overall

status,delayed,total_arrivals,overall_delay_rate
airline,,,
AM West,787,7225,0.108927
Alaska,501,3775,0.132715


## Conclusion

Based on the weighted overall delay rates, AM West demonstrates better
on-time performance than Alaska Airlines across the destinations analyzed.
Although Alaska shows lower delay rates on most individual routes, AM West
operates a significantly higher number of flights—particularly through
Phoenix—which lowers its overall weighted delay rate when total arrivals
are taken into account.

When examining performance by destination, Alaska generally performs better
on a route-by-route basis. However, the overall results differ once flight
volume is considered, highlighting how aggregated data can lead to different
conclusions than route-level comparisons.

### Limitations

This analysis is limited to five destinations and classifies arrivals only as
either on-time or delayed, without accounting for the length or cause of delays.
Additionally, the data represents a snapshot in time and may not reflect
seasonal trends or performance across a broader network of routes.